## Load data

In [187]:
import numpy as np
import scipy.io.arff as sparff

def loadData(filename):
    # reading the training data
    data, attribute = sparff.loadarff(filename)
    
    #getting size of attributes and instances
    numFeatures = len(attribute.names())-1
    numInstances = len(data)
    classLabel = attribute[attribute.names()[-1]]
    
    X, Y = [], []
    for i in range(numInstances):
        #print data[i][0]
        # convert labels to 0-1 encoding
        Y.append(classLabel[1].index(data[i][-1]))
        
        # create feature vector representation for each isntance
        featureVector = []
        for j in range(numFeatures):
            featureVector.append(data[i][j])
        X.append(featureVector)

    # get the number of possible values of each feature
#     numvals = []
#     for n in range(len(attribute.names())):
#         numvals.append(len(attribute[attribute.names()[n]][1]))
    npX = np.array(X)
    npY = np.array(Y)
    return npX, npY, attribute, numFeatures, numInstances, classLabel

In [242]:
X, Y, metadata, numFeatures, numInstances, classLabel = loadData('sonar.arff')

### Initialize weights and a helper function for sigmoid function

In [189]:
def sigmoidFunction(input):
    return np.divide(1.0,(np.add(1.0,np.exp(-input))))

def initializaWeights(numFeatures, numHidden):
    #print numHidden
    weightsHidden = []
    weightsOutput = []
    #weightsBias = []
    weightsHidden.append(np.random.uniform(-0.1, 0.1, (numFeatures, numHidden)))
    weightsOutput.append(np.random.uniform(-0.1, 0.1, numHidden))
    weightsBias.append(np.random.uniform(-0.1, 0.1, numFeatures))
    
    #print np.shape(weights[0])
    return weightsHidden, weightsOutput

In [190]:
weightsHidden, weightsOutput = initializaWeights(numFeatures, numFeatures)
# weightsHidden= np.array([[.15,.2],[.25,.3]]) 
# weightsOutput = np.array([[.4,.45]]) 
len(weightsHidden[0]), len(weightsOutput[0]), numFeatures

(60, 60, 60)

### train model

In [218]:
import math 
nEpochs = 10

def trainModel(X,Y,weightsHidden,weightsOutput):
    
    b1 = 0.35
    b2 = 0.6
    
    orderedIdx = np.random.permutation(len(Y))
    for i in orderedIdx:
        ## forward pass
        inputRow = np.array(X[i])
        #print inputRow

        ##hidden layer output calculation
        hiddenLayer = []
        for j in range(0,len(weightsHidden)):
            sumWeightsHidden = np.dot(inputRow,weightsHidden[j]) + b1
            hiddenOut = sigmoidFunction(sumWeightsHidden)
            #print sumWeightsHidden, hiddenOut
            hiddenLayer.append(hiddenOut)
        #print '\n'

        ##output layer output calculation
        sumWeightsOutput = np.dot(np.array(hiddenLayer),weightsOutput[0]) + b2
        outputOut = sigmoidFunction(sumWeightsOutput)
        #print sumWeightsOutput,outputOut

        
        error = 0.5 * math.pow((Y[i]-outputOut),2)
        #print error
        weightsHidden, weightsOutput = backProp(inputRow,hiddenLayer,outputOut,weightsOutput,weightsHidden)
        
    return weightsHidden, weightsOutput, error

In [219]:
def backProp(inputRow,hiddenLayer,outputOut,weightsOutput,weightsHidden):
    ## backpropagation
    lrate = 0.1

    #Output layer weight update
    delta_o = outputOut*(1-outputOut)*(Y[0]-outputOut)
    #print len(delta_o)
    wtsOut_gradient = np.multiply((lrate*delta_o),hiddenLayer)
    wtsOut_gradient
    weightsOutput = np.add (weightsOutput, wtsOut_gradient)

    #hidden layer weight update
    for j in range(0,len(weightsHidden)):
        delta_h = hiddenLayer[j]* (1-hiddenLayer[j])*delta_o*weightsOutput[0][j]
        #print len(delta_h)
        wtsHid_gradient = np.multiply((lrate*delta_h),inputRow)
        wtsHid_gradient
        weightsHidden [j] =np.add (weightsHidden[j], wtsHid_gradient)
    return weightsHidden, weightsOutput


# for e in range(nEpochs):
#     weightsHidden, weightsOutput, error = trainModel(X,Y,weightsHidden,weightsOutput)
#     print e, error
#     #print weightsHid, weightsOut

### test model

In [ ]:
def testModel(X,Y,weightsHidden,weightsOutput):
    

### Stratifying dataset

In [270]:
nfolds = 10
numNegInstance = list(Y).count(0)
numPosInstance = list(Y).count(1)
numNegInstance, numPosInstance
negIndices = np.where(Y==0)[0]
posIndices = np.where(Y==1)[0]

posInstanceInFold = (numPosInstance/nfolds)
negInstanceInFold = (numNegInstance/nfolds)

foldInstanceIndices = []
#foldLabelIndices = []
i= j = 0
for f in range(0,nfolds):
#     print posIndices[i:(i+posInstanceInFold)]
#     print negIndices[j:(j+negInstanceInFold)]
    foldInstanceIndices.append(np.append(posIndices[i:(i+posInstanceInFold)],negIndices[j:(j+negInstanceInFold)]))
    #foldLabelIndices.append(np.append(posIndices[i:(i+posInstanceInFold)],negIndices[j:(j+negInstanceInFold)]))
    i+=posInstanceInFold
    j+=negInstanceInFold
foldInstanceIndices

10

In [ ]:
prediction = []
for f in range(0,nfolds):
    tesFold = f
    for t in range(0,len(foldInstanceIndices)):
        if t==testFold:
            continue
            
        X_train = X[list(foldInstanceIndices[t]),:]
        Y_train = Y[list(foldInstanceIndices[t])]
        for e in range(nEpochs):
            weightsHidden, weightsOutput, error = trainModel(X_train,Y_train,weightsHidden,weightsOutput)
            print e, error
    X_test = X[list(foldInstanceIndices[testFold]),:]
    Y_test = Y[list(foldInstanceIndices[testFold])]    
    prediction.append(testModel(X_test,Y_test,weightsHidden,weightsOutput))

In [283]:
X[list(foldInstanceIndices[0]),:]

array([[ 0.0491,  0.0279,  0.0592, ...,  0.014 ,  0.0332,  0.0439],
       [ 0.1313,  0.2339,  0.3059, ...,  0.0127,  0.0178,  0.0231],
       [ 0.0201,  0.0423,  0.0554, ...,  0.0224,  0.019 ,  0.0096],
       ..., 
       [ 0.0317,  0.0956,  0.1321, ...,  0.0143,  0.0036,  0.0103],
       [ 0.0519,  0.0548,  0.0842, ...,  0.0047,  0.0048,  0.0053],
       [ 0.0223,  0.0375,  0.0484, ...,  0.0093,  0.0059,  0.0022]])

In [285]:
Y[[102, 103, 104, 105, 106, 107, 0, 1, 2, 3, 4, 5, 6, 7, 8]]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])